In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

%matplotlib inline
plt.style.use('ggplot')

In [ ]:
df = pd.read_csv('/Users/jenniferkaufman/Desktop/dsi_galvanize/capstones/capstone1/pretrial_services_success/ahAC21Q3.csv')

In [ ]:
df.head()

In [ ]:
df['CaseStart'] = pd.to_datetime(df['CaseStart'])
df['CaseEnd'] = pd.to_datetime(df['CaseEnd'])
df['length_of_supervision']=df['CaseEnd']-df['CaseStart'] #added column for length of supervision for further analysis

In [ ]:
collapsed_df= df.drop_duplicates(subset=['STAYID']) # dropped duplicates because each STAYID had multiple supervision level ids
collapsed_df

In [ ]:
collapsed_df.info()

In [ ]:
collapsed_df.describe()

Age <br>
Null Hypothesis:Pretrial services success rates do not vary by age.<br>
Alternate Hypothesis: Pretrial services success rates do vary by age.<br>
alpha=0.05<br>
statistical test=Pearson's chi-squared test ( χ2) ?<br>

Gender <br>
Null Hypothesis:Pretrial services success rates do not vary by gender.<br>
Alternate Hypothesis: Pretrial services success rates do vary by gender.<br>
alpha=0.05<br>
statistical test=<br>

Race <br>
Null Hypothesis:Pretrial services success rates do not vary by race.<br>
Alternate Hypothesis: Pretrial services success rates do vary by race.<br>
alpha=0.05<br>
statistical test=<br>

Ethnicity <br>
Null Hypothesis:Pretrial services success rates do not vary by ethnicity.<br>
Alternate Hypothesis: Pretrial services success rates do vary by ethnicity.<br>
alpha=0.05<br>
statistical test=<br>

Offense Type <br>
Null Hypothesis:Pretrial services success rates do not vary by offense type.<br>
Alternate Hypothesis: Pretrial services success rates do vary by offense type.<br>
alpha=0.05<br>
statistical test=<br>

Monitoring Type <br>
Null Hypothesis:Pretrial services success rates do not vary by monitoring type.<br>
Alternate Hypothesis: Pretrial services success rates do vary by monitoring type.<br>
alpha=0.05<br>
statistical test=<br>

Length of Supervision <br>
Null Hypothesis:Pretrial services success rates do not vary by length of supervision.<br>
Alternate Hypothesis: Pretrial services success rates do vary by length of supervision.<br>
alpha=0.05<br>
statistical test=<br>

Risk Category <br>
Null Hypothesis:Pretrial services success rates do vary by risk category.<br>
Alternate Hypothesis: Pretrial services success rates do not vary by risk category.<br>
alpha=0.05<br>
statistical test=<br>